In [1]:
using LatPhysReciprocal
using LatticePhysics
using HDF5

┌ Info: Recompiling stale cache file /home/jattig/.julia/compiled/v1.0/LatPhysReciprocal/VOqf5.ji for LatPhysReciprocal [7a511e88-ebe4-11e8-1bac-67d16a536c98]
└ @ Base loading.jl:1184
┌ Warning: Package LatPhysBase does not have HDF5 in its dependencies:
│ - If you have LatPhysBase checked out for development and have
│   added HDF5 as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LatPhysBase
└ Loading HDF5 into LatPhysBase from project dependency, future warnings for LatPhysBase are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for LatPhysReciprocal [7a511e88-ebe4-11e8-1bac-67d16a536c98]
│   exception = ErrorException("Required dependency LatPhysBase [eec5c15a-e8bd-11e8-0d23-6799ca40c963] failed to load from a cache file.")
└ @ Base loading.jl:963
┌ Info: Recompiling stale cache file /home/jattig/.julia/compiled/v1.0/LatPhysBase/Ef

# Reciprocal Paths

In [5]:
function saveReciprocalPath(
        rp :: RP,
        fn :: AbstractString,
        group :: AbstractString = "reciprocal_path"
        ;
        append :: Bool = false
    ) where {D,P<:ReciprocalPoint{D},RP<:ReciprocalPath{P}}
    
    # determine the mode based on if one wants to append stuff
    if append
        mode = "r+"
    else
        mode = "w"
    end

    # open the file in mode
    h5open(fn, mode) do file
        # create the group in which the bonds are saved
        group_path = g_create(file, group)
        # save the parameters
        attrs(group_path)["D"] = Int64(D)
        # save all Positions (D dimensions)
        if 0 < Int64(D)
            for n in 1:Int64(D)
                group_path["point_$(n)"] = Float64[point(p)[n] for p in rp]
            end
        end
        # save all labels
        group_path["label"]       = String[string(label(p)) for p in rp]
        group_path["label_LaTeX"] = String[string(labelLaTeX(p)) for p in rp]
    end

    # return nothing
    return nothing
end

saveReciprocalPath (generic function with 2 methods)

In [20]:
function loadReciprocalPath(
        ::Type{RPI},
        fn :: AbstractString,
        group :: AbstractString = "reciprocal_path"
    ) where {DI,PI<:ReciprocalPoint{DI},RPI<:Union{ReciprocalPath{PI},ReciprocalPath}}

    # read attribute data
    attr_data = h5readattr(fn, group)
    # determine D based on this
    D  = attr_data["D"]

    # load all remaining data
    rp_label       = h5read(fn, group*"/label")
    rp_label_LaTeX = h5read(fn, group*"/label_LaTeX")

    if D == 0
        rp_point = [Float64[] for i in 1:length(rp_label)]
    else
        rp_point_parts = [
            h5read(fn, group*"/point_"*string(j)) for j in 1:D
        ]
        rp_point = Vector{Float64}[Float64[rp_point_parts[j][i] for j in 1:D] for i in 1:length(rp_label)]
    end

    # create list of points in the path
    rpp = ReciprocalPoint{D}[
        newReciprocalPoint(ReciprocalPoint{D}, rp_point[i], rp_label[i], rp_label_LaTeX[i])
        for i in 1:length(rp_label)
    ]

    # return the path
    return newReciprocalPath(ReciprocalPath{ReciprocalPoint{D}}, rpp)
end


# convinience function for standard type
function loadReciprocalPath(
        fn :: AbstractString,
        group :: AbstractString = "reciprocal_path"
    )

    return loadReciprocalPath(ReciprocalPath, fn, group)
end

loadReciprocalPath (generic function with 4 methods)

In [27]:
recpath = getReciprocalPath(getUnitcellHyperhoneycomb(String,Int64), :K, :Gamma,:M,(:Mp,57),:Gamma)
saveReciprocalPath(recpath, "test.h5")
recpath

5-element ReciprocalPath{ReciprocalPoint{3}}:
 ReciprocalPoint{3} @[0.829031, 0.218166, -1.5708]: ("K" / L"K")   
 ReciprocalPoint{3} @[0.0, 0.0, 0.0]: ("Gamma" / L"\Gamma")        
 ReciprocalPoint{3} @[1.22173, -0.174533, -0.785398]: ("M" / L"M") 
 ReciprocalPoint{3} @[-0.174533, 1.22173, 0.785398]: ("M'" / L"M'")
 ReciprocalPoint{3} @[0.0, 0.0, 0.0]: ("Gamma" / L"\Gamma")        

In [28]:
loadReciprocalPath("test.h5")

5-element ReciprocalPath{ReciprocalPoint{3}}:
 ReciprocalPoint{3} @[0.829031, 0.218166, -1.5708]: ("K" / L"K")   
 ReciprocalPoint{3} @[0.0, 0.0, 0.0]: ("Gamma" / L"\Gamma")        
 ReciprocalPoint{3} @[1.22173, -0.174533, -0.785398]: ("M" / L"M") 
 ReciprocalPoint{3} @[-0.174533, 1.22173, 0.785398]: ("M'" / L"M'")
 ReciprocalPoint{3} @[0.0, 0.0, 0.0]: ("Gamma" / L"\Gamma")        